# Серия тиков с разной ценой, количество тиков N, расстояние во времени меньше T миллисекунд

In [13]:
from simple.ohlc import npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog, tickSpeed
from simple.backtest import npBacktestMarket, getProfit
import numpy as np
from simple.jurik import JRSX
from simple.numpy import rolling_apply

In [2]:
threshold = 4_000_000  # default resolution in microseconds
period = 60

In [3]:
C = np.load('data/ETHUSDT.2022-05-01.debounce.npz')['ETHUSDT'].view(np.recarray)
C = C[(np.datetime64('2022-05-08 00:00:01') < C.DT) & (C.DT < np.datetime64('2022-05-08 06:00'))]
C.Index = np.arange(len(C))
C[-5:]

rec.array([('2022-05-08T05:59:56.769999', 60618, 2537.025, 14058999,  4.698e+03, 6.557e+03, -1859.),
           ('2022-05-08T05:59:56.769999', 60619, 2537.135,        0,  2.020e+03, 2.020e+03,     0.),
           ('2022-05-08T05:59:56.769999', 60620, 2537.215,        0,  5.000e+00, 5.000e+00,     0.),
           ('2022-05-08T05:59:56.769999', 60621, 2537.245,        0,  2.600e+01, 2.600e+01,     0.),
           ('2022-05-08T05:59:56.864000', 60622, 2537.345,    94001, -6.733e+03, 1.046e+03, -7779.)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8')])

In [4]:
Buy = C[C.Size > 0]
Sell = C[C.Size < 0]
Sell[-5:]

rec.array([('2022-05-08T05:59:38.222000', 60606, 2537.005,    4000, -1.3070e+03,    4., -1.3110e+03),
           ('2022-05-08T05:59:39.714000', 60613, 2537.555, 1127000, -1.4246e+04,  717., -1.4963e+04),
           ('2022-05-08T05:59:42.684000', 60616, 2537.145,       0, -8.5000e+01,    0., -8.5000e+01),
           ('2022-05-08T05:59:42.711000', 60617, 2537.115,       0, -4.0000e+00,    0., -4.0000e+00),
           ('2022-05-08T05:59:56.864000', 60622, 2537.345,   94001, -6.7330e+03, 1046., -7.7790e+03)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8')])

In [5]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:00:00.643000', 2635.79003906, 2635.7800293 ),
           ('2022-05-08T00:00:00.743000', 2635.60009766, 2635.59008789),
           ('2022-05-08T00:00:00.844000', 2635.60009766, 2635.59008789),
           ('2022-05-08T00:00:00.944000', 2635.60009766, 2635.59008789),
           ('2022-05-08T00:00:01.044000', 2635.60009766, 2635.59008789)],
          dtype=[('DT', '<M8[us]'), ('A', '<f8'), ('B', '<f8')])

In [6]:
S[-5:]

rec.array([('2022-05-08T23:59:59.558999', 2519.7199707, 2519.70996094),
           ('2022-05-08T23:59:59.657999', 2519.7199707, 2519.70996094),
           ('2022-05-08T23:59:59.757999', 2519.7199707, 2519.70996094),
           ('2022-05-08T23:59:59.857999', 2519.7199707, 2519.70996094),
           ('2022-05-08T23:59:59.958000', 2519.7199707, 2519.70996094)],
          dtype=[('DT', '<M8[us]'), ('A', '<f8'), ('B', '<f8')])

In [7]:
K = npJoin(C.DT, S.DT)
len(K)

60623

In [8]:
# calculate sum of duration/volume by sliding window
def getSum(x):
    return np.sum(x).astype(int)

def getDelta(x):
    return x[-1] - x[0]

Duration = rolling_apply(getSum, period, C.Duration)
Volume = rolling_apply(getSum, period, C.Size)
Delta = rolling_apply(getDelta, period, C.Price)
len(Duration)

60623

In [9]:
# trade signal
s = 0
Signal = np.zeros_like(Duration)
for i in range(len(Signal)):
    if Duration[i] < threshold and C.Size[i] > 0:
        s = 1
    elif Duration[i] < threshold and C.Size[i] < 0:
        s = -1
    Signal[i] = s

Signal[-10:]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [10]:
D = npBacktestMarket(S.DT[K], S.A[K], S.B[K], Signal, 0)
D[-5:]

rec.array([(59854, '2022-05-08T05:48:40.817000', 2542.58007812, 2542.58508301, 59854, '2022-05-08T05:48:40.817000', 2542.59008789, 2542.58508301, -1.),
           (59855, '2022-05-08T05:48:41.316999', 2542.63989258, 2542.6348877 , 59857, '2022-05-08T05:48:41.517000', 2542.34008789, 2542.34509277,  1.),
           (59862, '2022-05-08T05:48:41.617000', 2541.98999023, 2541.99499512, 59864, '2022-05-08T05:48:41.716999', 2542.13989258, 2542.1348877 , -1.),
           (59864, '2022-05-08T05:48:41.716999', 2542.13989258, 2542.1348877 , 59869, '2022-05-08T05:48:44.318000', 2541.60009766, 2541.60510254,  1.),
           (59870, '2022-05-08T05:48:48.020000', 2541.87011719, 2541.8651123 , 60360, '2022-05-08T05:56:54.171000', 2536.69995117, 2536.70495605,  1.)],
          dtype=[('X0', '<i8'), ('T0', '<M8[us]'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<M8[us]'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [11]:
P = getProfit(D)
P[-5:]

rec.array([(59854, '2022-05-08T05:48:40.817000', -0.01000977, -0.        , 2.54258508, -2.55259485),
           (59857, '2022-05-08T05:48:41.517000', -0.29980469, -0.28979492, 2.54248999, -2.84229468),
           (59864, '2022-05-08T05:48:41.716999', -0.14990234, -0.13989258, 2.54206494, -2.69196729),
           (59869, '2022-05-08T05:48:44.318000', -0.53979492, -0.52978516, 2.54187   , -3.08166492),
           (60360, '2022-05-08T05:56:54.171000', -5.17016602, -5.16015625, 2.53928503, -7.70945105)],
          dtype=[('Index', '<i8'), ('DT', '<M8[us]'), ('RawPnL', '<f8'), ('MidPnL', '<f8'), ('Fee', '<f8'), ('Profit', '<f8')])

In [16]:
# indexed chart with bidask
fig = chartFigure(height=1200, rows=6,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Size), opacity=0.25),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size), opacity=0.25),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.2),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.2),

    #**chartTrades(D, tm=False),

    # Signal and duration indicators
    Speed=dict(x=C.Index, y=tickSpeed(C, period), color='green', row=2),
    LogSpeed=dict(x=C.Index, y=symlog(tickSpeed(C, period)), color='red', row=2, secondary_y=True, opacity=0.4),
    Duration=dict(x=C.Index, y=Duration, color='orange', row=3),
    Delta=dict(x=C.Index, y=Delta, color='blue', row=4),
    Signal=dict(x=C.Index, y=Signal, color='magenta', row=5, shape='hv'),
    RSX=dict(x=C.Index, y=JRSX(C.Price, period), color='cyan', row=6),

    # Profit
    RawPnL=dict(x=P.Index, y=P.RawPnL.cumsum(), secondary_y=True, width=4, color='gray', opacity=0.4, shape='hv', connectgaps=True),
    Profit=dict(x=P.Index, y=P.Profit.cumsum(), secondary_y=True, width=2, color='blue', opacity=0.3, shape='hv', connectgaps=True)
)

# styling
fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=2, spikesnap='cursor')
fig.update_traces(xaxis='x4')
fig.update_layout(legend=dict(x=0, y=1.05, bordercolor="lightgray", borderwidth=1))

fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …